In [1]:
### Imports
import gzip
from collections import defaultdict
import math
import numpy as np
import string
import random
import string
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import seaborn as sns
import pandas as pd
import os
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
### EVALUATION / METRICS
######################################
def convert_to_np_array(A):
    """
    If A is not already an array, convert it to an array
    """
    if not isinstance(A, np.ndarray): return np.array(A)
    else: return A

def get_MSE(A, B):
    """
    Given list A and list B:
    Return the mean squared error between A and B
    """
    return np.mean((convert_to_np_array(A) - convert_to_np_array(B))**2)

def inner(A, B):
    """
    Return the dot product between list A and list B
    """
    return np.dot(convert_to_np_array(A), convert_to_np_array(B))

def get_SSE(A, B):
    """
    Given list A and list B:
    Return the sum of squared errors between A and B
    """
    return np.sum((convert_to_np_array(A) - convert_to_np_array(B))**2)

def get_MAE(A,B):
    """
    Given list A and list B:
    Return the mean absolute error between A and B
    """

    errors = [abs(a - b) for a,b in zip(A,B)]
    return sum(errors) / len(errors)

def get_SE(A,B):
    """
    Given list A and list B:
    Return the squared error between each element
    """
    return (convert_to_np_array(A) - convert_to_np_array(B))**2

def get_accuracy(A,B):
    """
    Given list A and list B:
    Return the accuracy
    """
    return np.sum(convert_to_np_array(A) == convert_to_np_array(B)) / len(A)

def get_BER(y_actual, y_predicted):
    """
    "Return the balanced error rate between positive (1) and negative(0) instances
    """

    TP, FP, TN, FN = 0, 0, 0, 0
    n_pos, n_neg = 0, 0
    for actual, pred in zip(y_actual, y_predicted):
        if actual==1:
            n_pos += 1
            if actual==pred:
                TP += 1
            else:
                FN += 1
        else:
            n_neg += 1
            if actual==pred:
                TN += 1
            else:
                FP += 1
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
        
    return (1/2) * (FPR + FNR)

def get_errorMetrics_binary(y_actual, y_predicted, beta=1):
    """
    Return a set of error metrics between positive (1) and negative (0) instances
    This is valid for a binary class case
    Return a dictionary containing all calculated values
    """

    output = {}
    TP, FP, TN, FN = 0, 0, 0, 0
    n_pos, n_neg = 0, 0
    for actual, pred in zip(y_actual, y_predicted):
        if actual==1:
            n_pos += 1
            if actual==pred:
                TP += 1
            else:
                FN += 1
        else:
            n_neg += 1
            if actual==pred:
                TN += 1
            else:
                FP += 1
    ###
    TPR, FNR = TP / n_pos, FN / n_pos
    FPR, TNR = FP / n_neg, TN / n_neg
    prec = TP / (TP + FP)
    recall = TP / (TP + FN)
    output["TP"], output["FP"], output["TN"], output["FN"] = TP, FP, TN, FN
    output["TPR"], output["FPR"], output["FNR"], output["TNR"] = TPR, FPR, FNR, TNR
    output["precision"], output["recall"] = prec, recall
    output["BER"] = (1/2) * (FPR + FNR)
    output[f"F{beta}_Score"] = (1 + beta**2) * (prec * recall) / ((beta**2)*prec + recall)
    output["F_Score"] = 2 * (prec * recall) / (prec + recall)

    return output

In [3]:
### SIMILARITY FUNCTIONS
######################################
def jaccard_sim(A,B):
    """
    Return the Jaccard similarity between list A and list B
    """
    if not isinstance(A, set): A = set(A)
    if not isinstance(B, set): B = set(B)
    n_intersect = len(A.intersection(B))
    n_union = len(A.union(B))
    if n_union == 0: return 0

    return n_intersect / n_union

def cosine_sim_binary(A,B, denom_over_all=True):
    """
    Return the cosine similarity between set A and set B (Binary interactions)
    """
    if not isinstance(A, set): A = set(A)
    if not isinstance(B, set): B = set(B)
    n_intersect = len(A.intersection(B))

    if denom_over_all:
        total_interactions = np.sqrt(len(A) * len(B))
    else:
        total_interactions = n_intersect
    if total_interactions == 0:
        return 0
    return n_intersect / total_interactions

############# Design structures to record shared items
def cosine_sim(x_tuple, y_tuple, denom_over_all):
    """
    Calculate the cosine similarity between lists x and y
    Input are lists of tuples: [(id1, rating), (id2, rating), ...]
    """
    # Get shared items
    x_ids, y_ids = set(), set()
    x_ratings, y_ratings = [], []
    shared_ratings_x, shared_ratings_y = [], []
    shared_tuples_x, shared_tuples_y = [], []
    for tuple in x_tuple:
        x_ids.add(tuple[0])
        x_ratings.append(tuple[1])
    for tuple in y_tuple:
        y_ids.add(tuple[0])
        y_ratings.append(tuple[1])
    shared_ids = x_ids.intersection(y_ids)

    shared_tuples_x = [tuple for tuple in x_tuple if tuple[0] in shared_ids]
    shared_tuples_x.sort()
    shared_tuples_y = [tuple for tuple in y_tuple if tuple[0] in shared_ids]
    shared_tuples_y.sort()
    shared_ratings_x = [tuple[1] for tuple in shared_tuples_x]
    shared_ratings_y = [tuple[1] for tuple in shared_tuples_y]

    if denom_over_all:
        # Use all items in the denominator
        x_norm = np.sum([xi**2 for xi in x_ratings])
        y_norm = np.sum([yi**2 for yi in y_ratings])
    else:
        # Only use shared items in the denominator
        x_norm = np.sum([xi**2 for xi in shared_ratings_x])
        y_norm = np.sum([yi**2 for yi in shared_ratings_y])
    denom = np.sqrt(x_norm) * np.sqrt(y_norm)

    if denom == 0: return 0
    numer = sum([xi*yi for xi,yi in zip(shared_ratings_x, shared_ratings_y)])

    return numer / denom

def pearson_sim(x_tuple, y_tuple):
    """
    Calculate the pearson similarity between lists x and y
    Input are lists of tuples: [(id1, rating), (id2, rating), ...]
    Unlike Cosine sim, ONLY shared items can be considered
    If id1 or id2 is not in the relevant training data structure, use meanValue as its respective mean
    """
    # Unpack averages
    x_avgs = {tuple[0][0]:tuple[1] for tuple in x_tuple}
    y_avgs = {tuple[0][0]:tuple[1] for tuple in y_tuple}
    # Get shared items
    shared_ratings_x, shared_ratings_y = [], []
    shared_tuples_x, shared_tuples_y = [], []
    x_ids = {tuple[0][0] for tuple in x_tuple}
    y_ids = {tuple[0][0] for tuple in y_tuple}
    shared_ids = x_ids.intersection(y_ids)

    shared_tuples_x = [tuple[0] for tuple in x_tuple if tuple[0][0] in shared_ids]
    shared_tuples_x.sort()
    shared_tuples_y = [tuple[0] for tuple in y_tuple if tuple[0][0] in shared_ids]
    shared_tuples_y.sort()
    shared_ratings_x = [tuple[1] - x_avgs[tuple[0]] for tuple in shared_tuples_x] ### Pearson --> Subtract the mean from each value
    shared_ratings_y = [tuple[1] - y_avgs[tuple[0]] for tuple in shared_tuples_y]

    # Only use shared items in the denominator
    x_norm = np.sum([xi**2 for xi in shared_ratings_x])
    y_norm = np.sum([yi**2 for yi in shared_ratings_y])
    denom = np.sqrt(x_norm * y_norm)

    if denom == 0: return 0
    numer = sum([xi*yi for xi,yi in zip(shared_ratings_x, shared_ratings_y)])

    return numer / denom


In [4]:
### COLLABORATIVE FILTERING
######################################


def predictValue_bySim_devFromMean(user_id, item_id, sim_func, type, meanValue, value_bounds=None, denom_over_all=None, **kwargs):
    """
    Predict some value (e.g., rating) that the user (user_id) will give an item (item_id) based on
    the input simularity function (sim_func). However, instead of predicting the rating directly,
    predict the deviation from the global mean rating

    # Necessary Data Structures --> These are built using training data ONLY
    itemsPerUser: A dictionary containing the list of items each user interacted with and corresponding values
       ex: itemsPerUser[user1] = [(item1, 2), (item3, 1), (item5, 5), ...]
    usersPerItem: A dictionary containing the list of users that interacted with each item and corresponding values
       ex: usersPerItem[item1] = [(user1, 2), (user4, 2), (user27, 2), ...]
    itemAverages: A dictionary containing the mean value for each item
    userAverages: A dictionary containing the mean value for each user
    #--- value_bounds: The min/max values that can be outputted as typically there's a scale (e.g., 1-5 stars)
    #--- meanValue: The mean of all values in valueDict.values()

    # Gathering similarity weights:
    If type==0:
       # Predict the rating as a weighted sum of ratings that user_id has given to other items #
       For each item (item_id2) that user_id has interacted with (except for item_id):
          Calculate item_id2's similarity to item_id based on shared user interactions
          Track these values as similarity weights
    else:
       # Predict the rating as a weighted sum of ratings that other users have given to item_id #
       For each user (user_id2) that have interacted with item_id (except for user_id):
          Calculate user_id2's similarity to user_id based on shared item interactions
          Track these values as similarity weights
    """
    # Edge case 1: Return global mean value if user_id or item_id are unseen
    if (user_id not in itemsPerUser) or (item_id not in usersPerItem): return meanValue
    # Initialize variables
    if denom_over_all is None: denom_over_all = True
    if value_bounds is None: value_bounds = (-np.inf, np.inf)
    values, similarities = [], []

    if type == "item":
        # Predict the rating as a weighted combination of how other items rated by user_id were
        # rated by similar users
        # if user_id not in userAverages: return meanValue # Skip keys that are not in the dict
        avg_value = userAverages[user_id]
        for user_id2,value in usersPerItem[item_id]:
            if user_id2 == user_id: continue
            # if user_id2 not in userAverages: continue  # Skip keys that are not in the dict
            if sim_func == jaccard_sim:
                simset_user_id = {tuple[0] for tuple in itemsPerUser[user_id] if tuple[0] != item_id}
                simset_user_id2 = {tuple[0] for tuple in itemsPerUser[user_id2] if tuple[0] != item_id}
                similarities.append(sim_func(simset_user_id, simset_user_id2))
            elif sim_func == cosine_sim_binary:
                if denom_over_all is None: denom_over_all = True
                simset_user_id = {tuple[0] for tuple in itemsPerUser[user_id] if tuple[0] != item_id}
                simset_user_id2 = {tuple[0] for tuple in itemsPerUser[user_id2] if tuple[0] != item_id}
                similarities.append(sim_func(simset_user_id, simset_user_id2, denom_over_all))
            elif sim_func == cosine_sim:
                if denom_over_all is None: denom_over_all = True
                simset_user_id = {tuple for tuple in itemsPerUser[user_id] if tuple[0] != item_id}
                simset_user_id2 = {tuple for tuple in itemsPerUser[user_id2] if tuple[0] != item_id}
                similarities.append(sim_func(simset_user_id, simset_user_id2, denom_over_all))
            elif sim_func == pearson_sim:
                simset_user_id = {(tuple, itemAverages[tuple[0]]) for tuple in itemsPerUser[user_id] if tuple[0] != item_id}
                simset_user_id2 = {(tuple, itemAverages[tuple[0]]) for tuple in itemsPerUser[user_id2] if tuple[0] != item_id}
                similarities.append(sim_func(simset_user_id, simset_user_id2))
            else:
                # Sim function not programmed
                print("Invalid sim_func")
                return None
            values.append(value - userAverages[user_id2])
    else:
        # Predict user_id's rating of item_id based on a weighted combination of how other users who
        # rated item_id rated other items
        # if item_id not in itemAverages: return meanValue # Skip keys that are not in the dict
        avg_value = itemAverages[item_id]
        for item_id2,value in itemsPerUser[user_id]:
            if item_id2 == item_id: continue
            # if item_id2 not in itemAverages: continue # Skip keys that are not in the dict
            if sim_func == jaccard_sim:
                simset_item_id = {tuple[0] for tuple in usersPerItem[item_id] if tuple[0] != user_id}
                simset_item_id2 = {tuple[0] for tuple in usersPerItem[item_id2] if tuple[0] != user_id}
                similarities.append(sim_func(simset_item_id, simset_item_id2))
            elif sim_func == cosine_sim_binary:
                if denom_over_all is None: denom_over_all = True
                simset_item_id = {tuple[0] for tuple in usersPerItem[item_id] if tuple[0] != user_id}
                simset_item_id2 = {tuple[0] for tuple in usersPerItem[item_id2] if tuple[0] != user_id}
                similarities.append(sim_func(simset_item_id, simset_item_id2, denom_over_all))
            elif sim_func == cosine_sim:
                if denom_over_all is None: denom_over_all = True
                simset_item_id = {tuple for tuple in usersPerItem[item_id] if tuple[0] != user_id}
                simset_item_id2 = {tuple for tuple in usersPerItem[item_id2] if tuple[0] != user_id}
                similarities.append(sim_func(simset_item_id, simset_item_id2, denom_over_all))
            elif sim_func == pearson_sim:
                simset_item_id = {(tuple, userAverages[tuple[0]]) for tuple in usersPerItem[item_id] if tuple[0] != user_id}
                simset_item_id2 = {(tuple, userAverages[tuple[0]]) for tuple in usersPerItem[item_id2] if tuple[0] != user_id}
                similarities.append(sim_func(simset_item_id, simset_item_id2))
            else:
                # Sim function not programmed
                print("Invalid sim_func")
                return None
            values.append(value - itemAverages[item_id2])
    # Edge case 2: Return global mean value if there are no similar items
    if np.sum(similarities) == 0: return meanValue

    numerator = np.sum([value*sim for value,sim in zip(values, similarities)])
    denominator = np.sum(similarities)
    output = avg_value + (numerator / denominator)
    if output < value_bounds[0]: return value_bounds[0]
    if output > value_bounds[1]: return value_bounds[1]

    return output

In [5]:
### LATENT FACTOR MODEL
######################################
def initialize_params(ratingDict, itemsPerUser, usersPerItem, k, init_bounds):
    """
    Return the initialized parameters
    """
    user_count, item_count = len(itemsPerUser), len(usersPerItem)
    lower, upper = init_bounds
    alpha = np.mean([i for i in ratingDict.values()])
    user_bias = dict(zip(users, [random.uniform(lower,upper) for i in range(user_count)]))
    item_bias = dict(zip(items, [random.uniform(lower,upper) for i in range(item_count)]))
    user_gamma = dict(zip(users, [[random.uniform(lower,upper) for ki in range(k)] for i in range(user_count)]))
    item_gamma = dict(zip(items, [[random.uniform(lower,upper) for ki in range(k)] for i in range(item_count)]))

    return (alpha, user_bias, item_bias, user_gamma, item_gamma)

def get_lfm_defaults(theta):
    """
    Calculate average values/vectors to use as default values for the latent factor predictions
    """
    alpha, user_bias, item_bias, user_gamma, item_gamma = theta
    avg_params = {}
    avg_params["avg_user_bias"] = np.mean(np.array([i for i in user_bias.values()]))
    avg_params["avg_item_bias"] = np.mean(np.array([i for i in item_bias.values()]))
    avg_params["avg_user_gamma"] = np.mean(np.array([values for values in user_gamma.values()]), axis=0)
    avg_params["avg_item_gamma"] = np.mean(np.array([values for values in item_gamma.values()]), axis=0)

    return avg_params

def predict_latent_factor(u_id, item_id, theta, value_bounds = None, avg_params = None):
    """
    Return the prediction based on u_id and item_id
    If u_id is not in itemsPerUser --> Use the average gamma vector
    Repeat this process for if item_id is not in usersPerItem

    Bound the output by the min and max of the possible values 
    (ex: Model shouldn't exceed 5 when the scale is 5)
    """
    if value_bounds is None: value_bounds = (-np.inf, np.inf)
    alpha, user_bias, item_bias, user_gamma, item_gamma = theta
    for i,(gamma_vec) in enumerate(user_gamma.values()):
        if i > 0: break
        k = len(gamma_vec)

    if u_id in user_bias: u_bias = user_bias[u_id]
    else:
        if (avg_params is not None) and (item_id in item_bias): u_bias = avg_params["avg_user_bias"]
        else: u_bias = 0

    if item_id in item_bias: i_bias = item_bias[item_id]
    else:
        if (avg_params is not None) and (u_id in user_bias): i_bias = avg_params["avg_item_bias"]
        else: i_bias = 0

    if u_id in user_gamma: u_gamma = np.array(user_gamma[u_id])
    else:
        if (avg_params is not None) and (item_id in item_gamma): u_gamma = avg_params["avg_user_gamma"]
        else: u_gamma = [0]*k

    if item_id in item_gamma: i_gamma = np.array(item_gamma[item_id])
    else:
        if (avg_params is not None) and (u_id in user_gamma): i_gamma = avg_params["avg_item_gamma"]
        else: i_gamma = [0]*k

    value = alpha + u_bias + i_bias + np.dot(u_gamma, i_gamma)
    if value < value_bounds[0]: return value_bounds[0]
    if value > value_bounds[1]: return value_bounds[1]

    return value

def get_cost(theta, lambda_bias, lambda_gamma, Xtrain, ytrain, k, value_bounds=None):
    """
    Calculate the cost for the given theta parameters
    Xtrain must be of form [(user, item), (user, item), ...]
    """
    alpha, user_bias, item_bias, user_gamma, item_gamma = theta
    # Predict using the current theta values
    predictions = np.array([predict_latent_factor(tuple[0], tuple[1], theta, value_bounds) for tuple in Xtrain])
    # Calculate SSE + regularization
    cost = get_SSE(predictions, ytrain)
    cost += lambda_bias * np.sum(np.array([val**2 for val in user_bias.values()]))
    cost += lambda_bias * np.sum(np.array([val**2 for val in item_bias.values()]))
    cost += lambda_gamma * np.sum(np.array([inner(gam, gam) for gam in user_gamma.values()]))
    cost += lambda_gamma * np.sum(np.array([inner(gam, gam) for gam in item_gamma.values()]))

    return cost

def update_params(theta, lambda_bias, lambda_gamma, Xtrain, ytrain, fix_value):
    """
    Update parameters based on how well they predict in their CURRENT states
    Coordinate descent instead of gradient descent for faster convergence
    ###
    Latent factor model
         Fix user_gamma --> iterate and update alpha, user_bias, item_gamma
         Fix item_gamma --> iterate and update alpha, user_bias, user_gamma
    Fix user_gamma... Fix item_gamma...
    Repeat the above until model have converged
    """
    alpha, user_bias, item_bias, user_gamma, item_gamma = theta
    # Iterate through the alpha and bias params
    if (fix_value == 0) or (fix_value == 4):
        # Calculate the new alpha
        alpha_sum = 0
        for tuple, rating in zip(Xtrain, ytrain):
            u_id, item_id = tuple
            alpha_sum += (rating - (user_bias[u_id] + item_bias[item_id] + inner(user_gamma[u_id], item_gamma[item_id])))
        dalpha = alpha_sum / N
        alpha = dalpha.copy()
    elif (fix_value == 1) or (fix_value == 5):
        # Calculate the new user bias
        duser_bias = {}
        for u_id in users:
            user_sum = 0
            users_item_count = len(itemsPerUser[u_id])
            for tuple in itemsPerUser[u_id]:
                item_id, rating = tuple
                user_sum += (rating - (alpha + item_bias[item_id] + inner(user_gamma[u_id], item_gamma[item_id])))
            duser_bias[u_id] = (user_sum * (1 / (users_item_count + lambda_bias)))
        user_bias = duser_bias.copy()
    elif (fix_value == 2) or (fix_value == 6):
        # Calculate the new item bias
        ditem_bias = {}
        for item_id in items:
            item_sum = 0
            items_user_count = len(usersPerItem[item_id])
            for tuple in usersPerItem[item_id]:
                u_id, rating = tuple
                item_sum += (rating - (alpha + user_bias[u_id] + inner(user_gamma[u_id], item_gamma[item_id])))
            ditem_bias[item_id] = (item_sum * (1 / (items_user_count + lambda_bias)))
        item_bias = ditem_bias.copy()
    elif fix_value == 3:
        # Calculate the new user gamma
        duser_gamma = {}
        for u_id in users:
            user_sums = [0] * k
            i_gamma_k_sqrd_sums = [0] * k
            users_item_count = len(itemsPerUser[u_id])
            for tuple in itemsPerUser[u_id]:
                item_id, rating = tuple
                # Update each user's gamma value
                for ki in range(k):
                    i_gamma_k = item_gamma[item_id][ki]
                    i_gamma_k_sqrd_sums[ki] += i_gamma_k**2
                    user_sums[ki] = i_gamma_k * (rating - (alpha + user_bias[u_id] + item_bias[item_id]))
            duser_gamma[u_id] = [user_sums[ki] * (1 / (users_item_count*i_gamma_k_sqrd_sums[ki] + lambda_gamma)) for ki in range(k)]
        user_gamma = duser_gamma.copy()
    elif fix_value == 7:
         # Calculate the new item gamma
        ditem_gamma = {}
        for item_id in items:
            item_sums = [0] * k
            u_gamma_k_sqrd_sums = [0] * k
            items_user_count = len(usersPerItem[item_id])
            for tuple in usersPerItem[item_id]:
                u_id, rating = tuple
                # Update each item's gamma value
                for ki in range(k):
                    u_gamma_k = user_gamma[u_id][ki]
                    u_gamma_k_sqrd_sums[ki] += u_gamma_k**2
                    item_sums[ki] = u_gamma_k * (rating - (alpha + user_bias[u_id] + item_bias[item_id]))
            ditem_gamma[item_id] = [item_sums[ki] * (1 / (items_user_count*u_gamma_k_sqrd_sums[ki] + lambda_gamma)) for ki in range(k)]
        item_gamma = ditem_gamma.copy()

    return (alpha, user_bias, item_bias, user_gamma, item_gamma)

def fit_parameters(Xtrain, ytrain, theta, ep=0.0005, iter_limit=200, quiet=True, value_bounds=(-np.inf, np.inf), check_every=10, **kwargs):
    """
    Fit the parameters until convergence (when difference in cost is less than ep)
    Arguments packed into **kwargs:
    lambda_bias --> Regularization parameter for the user/item biases
    lambda_gamma --> Regularization parameter for the user/item gamma matrix
    k --> Number of latent parameters to use per user/item vector
    ep --> The threshold for early stopping between mse checks
    iter_limit --> The maximum number of iterations allowed
    value_bounds --> The expected range of values expected to be outputted by the model
    check_every --> Calculate the cost/mse whenever iter_count is perfectly divisible by check_every (ex: check_every=10 means check every 10 iterations)
    """
    last_mse, last_cost = np.inf, np.inf
    best_params = [theta, last_mse, last_cost]

    iter_count = 0
    while True:
        # Track iteration count as well as which parameter to update (fix_value)
        iter_count += 1
        fix_value = (iter_count - 1) % 8

        # Update theta
        theta = update_params(theta, lambda_bias, lambda_gamma, Xtrain, ytrain, fix_value)

        # Every few iterations, check cost and mse
        if (iter_count % check_every == 0):
            cost = get_cost(theta, lambda_bias, lambda_gamma, Xtrain, ytrain, k, value_bounds)
            predictions = np.array([predict_latent_factor(tuple[0], tuple[1], theta, value_bounds) for tuple in Xtrain])
            mse = get_MSE(predictions, ytrain)
            if not quiet:
                print(f"Iteration {iter_count}: Cost = {cost}, Train MSE = {mse}")
            ### Check cost/mse
            # Save current params as best_params if the mse is less than the last mse
            if mse < best_params[1]:
                best_params = [theta, mse, cost]
                # If the difference in cost is less than ep, stop iterating
                if last_mse - mse > ep:
                    last_mse = mse
                    last_cost = cost
                else:
                    if not quiet:
                        print(f"Convergence after {iter_count} iterations: Cost = {cost}, Train MSE = {mse}")
                    break
            # Check if cost is too high - sometimes the algorithm diverges
            if mse > 5000:
                theta, mse, cost = best_params
                if not quiet:
                    print(f"Training MSE too high: Best Train MSE = {best_params[1]}")
                break
        # If the iteration limit is reached, stop and return the best parameters
        if iter_count > iter_limit:
            theta, mse, cost = best_params
            if not quiet:
                print(f"Iteration limit reached after {iter_count} iterations: Best Train MSE = {best_params[1]}")
            break

    return (theta, cost, mse)


In [6]:
### Popularity-based Baseline
#
# Rank the items in the dataset based on popularity --> If the item is popular, predict 1, else 0

def get_item_popularities(df, col):
    # Define popularity: The items which have the highest playtime per person ratios weighted by number of plays
    item_playCounts = df.groupby("item_id").agg("sum")[col]
    item_playCountsBinary = df.groupby("item_id").agg("sum")["playtime_binary"]
    item_userCounts = df.groupby("item_id").count()["user_id"]

    item_playsPerUser = (item_playCountsBinary / item_userCounts).sort_values(ascending=True)
    # item_playsPerUser = (item_playCounts).sort_values(ascending=True)

    return pd.DataFrame({"pop_values":item_playsPerUser, "pop_sums":item_playsPerUser.cumsum()})

def get_most_popular_items(df, cutoff):
    pop_cutoff = df["pop_values"].max() * cutoff

    return list(df[df["pop_values"] > pop_cutoff].index)

def predict_by_popularity(X, pop_items):
    if isinstance(X, tuple):
        if X[1] in pop_items: return 1
        else: return 0
    else:
        output = pd.Series([0 for i in range(len(X))])
        output[X["item_id"].isin(pop_items)] = 1
        return output

#####

In [7]:
### OTHER HELPFUL FUNCTIONS
######################################
def get_rec_structs(train_data):
    """
    Extract stats used for creating the classifier features
    Input is (user_id, item_id, value), ...
    Typically value is rating, but can be other things (e.g., hours played)

    itemsPerUser: Records each item in the training set that each user interacted with (along with the corresponding value)
    usersPerItem: Records each user in the training set that each item interacted with (along with the corresponding value)
    valueDict: Records the value for each (user, item) tuple
    userAverages: Gives the average value for each user
    itemAverages: Gives the average value for each item
    """
    ### Record which items each user interacted with and which users interacted with which item
    itemsPerUser = defaultdict(list)
    usersPerItem = defaultdict(list)
    valueDict = {}
    for u,b,v in train_data:
        itemsPerUser[u].append((b,v))
        usersPerItem[b].append((u,v))
        valueDict[(u,b)] = v

    ### Calculate user and item average ratings
    userAverages = {}
    itemAverages = {}
    for u,tuples in itemsPerUser.items():
        values = [value for item,value in tuples]
        # values = [value for item,value in tuples if value != 0]
        # if len(values) == 0: continue
        userAverages[u] = sum(values) / len(values)
    for i,tuples in usersPerItem.items():
        values = [value for user,value in tuples]
        # values = [value for user,value in tuples if value != 0]
        # if len(values) == 0: continue
        itemAverages[i] = sum(values) / len(values)

    rec_structs = {"itemsPerUser":itemsPerUser,
                  "usersPerItem":usersPerItem,
                  "valueDict":valueDict,
                  "userAverages":userAverages,
                  "itemAverages":itemAverages,}

    return rec_structs

def unpack_rec_structs(rec_structs):
    """
    Take the input recommender_structs and return the itemized contents
    """
    itemsPerUser = rec_structs["itemsPerUser"]
    usersPerItem = rec_structs["usersPerItem"]
    valueDict = rec_structs["valueDict"]
    userAverages = rec_structs["userAverages"]
    itemAverages = rec_structs["itemAverages"]

    return (itemsPerUser, usersPerItem, valueDict, userAverages, itemAverages)

### Functions to read files (From homework stubs)
def readGz(path):
    output = []
    for l in gzip.open(path, mode = 'rt', encoding = "utf-8"):
        output.append(eval(l))
    return output

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [8]:
%%time
### PREPROCESS THE DATA
seed = 100
interaction_filepath = "data/playtime_df.csv"
playtime_df = pd.read_csv(interaction_filepath).sample(frac=1, random_state=seed)

CPU times: total: 281 ms
Wall time: 292 ms


In [9]:
# Split the data into train-test-validation
n_interactions = len(playtime_df)
train_df = playtime_df.sample(frac=0.8, random_state=seed).reset_index(drop=True)
test_df = playtime_df[~playtime_df.index.isin(train_df.index)].reset_index(drop=True)
valid_df = train_df.sample(frac=0.2, random_state=seed).reset_index(drop=True)
train_df = train_df[~train_df.index.isin(valid_df.index)].reset_index(drop=True)
train_df2 = pd.concat([train_df, valid_df], axis=0).reset_index(drop=True)

print(f"# rows in train_df: {len(train_df)}")
print(f"# rows in valid_df: {len(valid_df)}")
print(f"# rows in train_df2: {len(train_df2)}")
print(f"# rows in test_df: {len(test_df)}")

test_df.describe()

# rows in train_df: 541528
# rows in valid_df: 135382
# rows in train_df2: 676910
# rows in test_df: 169227


,item_id,playtime,recent_playtime,playtime_log,recent_playtime_log,playtime_binary,recent_playtime_binary
count,169227.000000,169227.000000,169227.000000,169227.000000,169227.000000,169227.000000,169227.000000
mean,180278.419951,950.476951,8.591755,3.306487,0.110422,0.632245,0.026237
std,132440.047426,5194.154759,139.282759,3.031033,0.744192,0.482196,0.159840
min,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35140.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,214970.000000,32.000000,0.000000,3.496508,0.000000,1.000000,0.000000
75%,269390.000000,340.000000,0.000000,5.831882,0.000000,1.000000,0.000000
max,530720.000000,361496.000000,16166.000000,12.798009,9.690727,1.000000,1.000000


In [10]:
### Save train/valid/train2/test dfs for other models
train_df.to_csv("data/train_df.csv", index=False)
valid_df.to_csv("data/valid_df.csv", index=False)
train_df2.to_csv("data/train_df2.csv", index=False)
test_df.to_csv("data/test_df.csv", index=False)

In [11]:
# Further split the data into X/y pairs for convenience
class_col = "playtime_binary" # This what we want to predict
X_train, y_train = train_df[["user_id", "item_id"]], train_df[class_col]
X_valid, y_valid = valid_df[["user_id", "item_id"]], valid_df[class_col]
X_test, y_test = test_df[["user_id", "item_id"]], test_df[class_col]
X_train2, y_train2 = train_df2[["user_id", "item_id"]], train_df2[class_col]

In [12]:
test_df_results = test_df[["user_id", "item_id", "playtime", "playtime_log"]].copy()

### BASELINE MODEL

In [13]:
t0 = time.time()

In [14]:
%%time
# Always predict the mean
playtime_cols = ["playtime", "playtime_log"]
playtime_means = train_df2[playtime_cols].mean(axis=0)
#Always predict the median
playtime_medians = train_df2[playtime_cols].median(axis=0)

CPU times: total: 15.6 ms
Wall time: 19.1 ms


In [15]:
%%time
# Get predictions from the unscaled variables
test_df["playtime_mean_pred"] = playtime_means.loc["playtime"]
test_df["playtimeLog_mean_pred"] = playtime_means.loc["playtime_log"]

test_df["playtime_median_pred"] = playtime_medians.loc["playtime"]
test_df["playtimeLog_median_pred"] = playtime_medians.loc["playtime_log"]

test_df

CPU times: total: 0 ns
Wall time: 1.5 ms


,user_id,item_id,playtime,recent_playtime,playtime_log,recent_playtime_log,playtime_binary,recent_playtime_binary,playtime_mean_pred,playtimeLog_mean_pred,playtime_median_pred,playtimeLog_median_pred
0,u5084,31130,0,0,0.000000,0.000000,0,0,969.703293,3.30626,32.0,3.496508
1,u11861,233270,715,0,6.573680,0.000000,1,0,969.703293,3.30626,32.0,3.496508
2,u4949,220200,232,0,5.451038,0.000000,1,0,969.703293,3.30626,32.0,3.496508
3,u1808,4000,231,0,5.446737,0.000000,1,0,969.703293,3.30626,32.0,3.496508
4,u5401,1280,0,0,0.000000,0.000000,0,0,969.703293,3.30626,32.0,3.496508
...,...,...,...,...,...,...,...,...,...,...,...,...
169222,u11442,238960,20,0,3.044522,0.000000,1,0,969.703293,3.30626,32.0,3.496508
169223,u12710,4000,12936,5,9.467847,1.791759,1,1,969.703293,3.30626,32.0,3.496508
169224,u10496,8190,207,0,5.337538,0.000000,1,0,969.703293,3.30626,32.0,3.496508
169225,u6112,48700,0,0,0.000000,0.000000,0,0,969.703293,3.30626,32.0,3.496508


In [16]:
# Get MSE on regular playtime
mse_playtime = get_MSE(test_df["playtime"], test_df["playtime_mean_pred"])
mse_playtimeLog = get_MSE(test_df["playtime"], np.exp(test_df["playtimeLog_mean_pred"]) - 1)

print("MEANS")
print(f"MSE based on regular playtime scale: {mse_playtime}")
print(f"MSE based on regular playtime scale: {mse_playtimeLog}")

# Get MSE on regular playtime
mse_playtime = get_MSE(test_df["playtime"], test_df["playtime_median_pred"])
mse_playtimeLog = get_MSE(test_df["playtime"], np.exp(test_df["playtimeLog_median_pred"]) - 1)

print("MEDIANS")
print(f"MSE based on regular playtime scale: {mse_playtime}")
print(f"MSE based on regular playtime scale: {mse_playtimeLog}")

MEANS
MSE based on regular playtime scale: 26979453.8886885
MSE based on regular playtime scale: 27833218.881216425
MEDIANS
MSE based on regular playtime scale: 27822684.146129165
MSE based on regular playtime scale: 27822684.146129165


In [17]:
# Get MSE on log playtime
mse_playtime2 = get_MSE(test_df["playtime_log"], np.log(test_df["playtime_mean_pred"] + 1))
mse_playtimeLog2 = get_MSE(test_df["playtime_log"], test_df["playtimeLog_mean_pred"])

print("MEANS")
print(f"MSE based on log playtime scale: {mse_playtime2}")
print(f"MSE based on log playtime scale: {mse_playtimeLog2}")

# Get MSE on log playtime
mse_playtime2 = get_MSE(test_df["playtime_log"], np.log(test_df["playtime_median_pred"] + 1))
mse_playtimeLog2 = get_MSE(test_df["playtime_log"], test_df["playtimeLog_median_pred"])

print("MEDIANS")
print(f"MSE based on log playtime scale: {mse_playtime2}")
print(f"MSE based on log playtime scale: {mse_playtimeLog2}")

MEANS
MSE based on log playtime scale: 21.94296292972277
MSE based on log playtime scale: 9.187107397527571
MEDIANS
MSE based on log playtime scale: 9.223215240818883
MSE based on log playtime scale: 9.223215240818883


In [18]:
# Get MAE on regular playtime
mae_playtime = get_MAE(test_df["playtime"], test_df["playtime_mean_pred"])
mae_playtimeLog = get_MAE(test_df["playtime"], np.exp(test_df["playtimeLog_mean_pred"]) - 1)

print("MEANS")
print(f"MAE based on regular playtime scale: {mae_playtime}")
print(f"MAE based on regular playtime scale: {mae_playtimeLog}")

# Get MAE on regular playtime
mae_playtime = get_MAE(test_df["playtime"], test_df["playtime_median_pred"])
mae_playtimeLog = get_MAE(test_df["playtime"], np.exp(test_df["playtimeLog_median_pred"]) - 1)

print("MEDIANS")
print(f"MAE based on regular playtime scale: {mae_playtime}")
print(f"MAE based on regular playtime scale: {mae_playtimeLog}")

MEANS
MAE based on regular playtime scale: 1446.5152666476665
MAE based on regular playtime scale: 947.326055480168
MEDIANS
MAE based on regular playtime scale: 947.2356125204607
MAE based on regular playtime scale: 947.2356125204607


In [19]:
# Get MSE on log playtime
mae_playtime2 = get_MAE(test_df["playtime_log"], np.log(test_df["playtime_mean_pred"] + 1))
mae_playtimeLog2 = get_MAE(test_df["playtime_log"], test_df["playtimeLog_mean_pred"])

print("MEANS")
print(f"MAE based on regular playtime scale: {mae_playtime2}")
print(f"MAE based on regular playtime scale: {mae_playtimeLog2}")

# Get MSE on log playtime
mae_playtime2 = get_MAE(test_df["playtime_log"], np.log(test_df["playtime_median_pred"] + 1))
mae_playtimeLog2 = get_MAE(test_df["playtime_log"], test_df["playtimeLog_median_pred"])

print("MEDIANS")
print(f"MAE based on regular playtime scale: {mae_playtime2}")
print(f"MAE based on regular playtime scale: {mae_playtimeLog2}")

MEANS
MAE based on regular playtime scale: 3.893728328812909
MAE based on regular playtime scale: 2.7169138908933355
MEDIANS
MAE based on regular playtime scale: 2.713820254040632
MAE based on regular playtime scale: 2.713820254040632


In [20]:
t_elapsed = time.time() - t0
print(f"Time elapsed to train model = {t_elapsed // 60} min and {t_elapsed % 60}s ({t_elapsed}s)")

Time elapsed to train model = 0.0 min and 0.3052687644958496s (0.3052687644958496s)


In [21]:
test_df

,user_id,item_id,playtime,recent_playtime,playtime_log,recent_playtime_log,playtime_binary,recent_playtime_binary,playtime_mean_pred,playtimeLog_mean_pred,playtime_median_pred,playtimeLog_median_pred
0,u5084,31130,0,0,0.000000,0.000000,0,0,969.703293,3.30626,32.0,3.496508
1,u11861,233270,715,0,6.573680,0.000000,1,0,969.703293,3.30626,32.0,3.496508
2,u4949,220200,232,0,5.451038,0.000000,1,0,969.703293,3.30626,32.0,3.496508
3,u1808,4000,231,0,5.446737,0.000000,1,0,969.703293,3.30626,32.0,3.496508
4,u5401,1280,0,0,0.000000,0.000000,0,0,969.703293,3.30626,32.0,3.496508
...,...,...,...,...,...,...,...,...,...,...,...,...
169222,u11442,238960,20,0,3.044522,0.000000,1,0,969.703293,3.30626,32.0,3.496508
169223,u12710,4000,12936,5,9.467847,1.791759,1,1,969.703293,3.30626,32.0,3.496508
169224,u10496,8190,207,0,5.337538,0.000000,1,0,969.703293,3.30626,32.0,3.496508
169225,u6112,48700,0,0,0.000000,0.000000,0,0,969.703293,3.30626,32.0,3.496508


In [22]:
# Save the one with the best results
test_df_results["baseline_mean_pred"] = test_df["playtimeLog_mean_pred"]
test_df_results["baseline_median_pred"] = test_df["playtimeLog_median_pred"]

test_df_results

,user_id,item_id,playtime,playtime_log,baseline_mean_pred,baseline_median_pred
0,u5084,31130,0,0.000000,3.30626,3.496508
1,u11861,233270,715,6.573680,3.30626,3.496508
2,u4949,220200,232,5.451038,3.30626,3.496508
3,u1808,4000,231,5.446737,3.30626,3.496508
4,u5401,1280,0,0.000000,3.30626,3.496508
...,...,...,...,...,...,...
169222,u11442,238960,20,3.044522,3.30626,3.496508
169223,u12710,4000,12936,9.467847,3.30626,3.496508
169224,u10496,8190,207,5.337538,3.30626,3.496508
169225,u6112,48700,0,0.000000,3.30626,3.496508


In [23]:
test_df_results.to_csv("data/test_results_baseline.csv", index=False)